<a href="https://colab.research.google.com/github/ZwL6/MZT/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=d955569559477ea5a280cd155b6859c441fc028a303d4056751b159dfd4f42c7
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:a

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
# Get cities file
!wget https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json



--2023-06-30 13:27:34--  https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15829 (15K) [text/plain]
Saving to: ‘cities.json’

cities.json         100%[===================>]  15.46K  --.-KB/s    in 0s      

2023-06-30 13:27:34 (142 MB/s) - ‘cities.json’ saved [15829/15829]



In [4]:
#making standerd route
import json
import random

# Set seed
random.seed(1)
with open('cities.json', 'r') as file:
    data = json.load(file)

cities = [d['city'] for d in data]

merchandise = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

routes = []

for i in range(1000):
    route = {}
    route['id'] = i

    # Generate a list of non-repeating random cities
    city_sample = random.sample(cities, 3)

    # Randomly generate the number of merchandise
    num_merchandise = random.randint(1, 10)

    # Randomly select merchandise types
    merchandise_sample = random.sample(merchandise, random.randint(3, 5))

    # Build the route dictionary format
    route['route'] = []
    for j in range(2):
        route_info = {}
        route_info['from'] = city_sample[j]
        route_info['to'] = city_sample[j+1]

        # Generate random merchandise quantities for each route
        merchandise_dict = {}
        for m in merchandise_sample:
            merchandise_dict[m] = random.randint(1, num_merchandise)

        route_info['merchandise'] = merchandise_dict

        route['route'].append(route_info)

    routes.append(route)

# Write the route list to a JSON file without corrupt records
valid_routes = [r for r in routes if 'route' in r and len(r['route']) == 2]
with open('routes.json', 'w') as file:
    json.dump(routes, file, indent=4)


In [5]:
# Download routes
# from google.colab import files
# files.download('routes.json')


In [6]:
# making actual route

import json
import random
import copy

# Set seed
random.seed(2)

# Read the JSON file
with open('cities.json', 'r') as file:
    data = json.load(file)

# Extract city names
cities = [item['city'] for item in data]

# Define merchandise types
merchandise_types = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

with open('routes.json', 'r') as file:
    routes = json.load(file)

act_routes = []

for route in routes:
    if len(route['route']) < 2:
        continue  # Skip routes with less than two cities

    for _ in range(10):
        act_route = copy.deepcopy(route)  # Use deepcopy to avoid modifying the original route

        # Randomly select a city to modify
        modified_city_index = random.randint(0, len(act_route['route']) - 1)
        modified_city = act_route['route'][modified_city_index]

        # Randomly decide whether to modify city
        if random.choice([True, False]):
            # Modify the selected city
            new_city = random.choice(cities)
            modified_city['from'] = new_city

            # If the modified city is the second city, modify it throughout the route
            if modified_city_index == 1:
                # Replace the modified city with a new city
                act_route['route'][modified_city_index]['from'] = new_city
                # Connect the cities before and after the replaced city
                act_route['route'][modified_city_index - 1]['to'] = new_city
                if modified_city_index + 1 < len(act_route['route']):
                    act_route['route'][modified_city_index]['to'] = act_route['route'][modified_city_index + 1]['from']

        # Randomly modify merchandise
        for city_route in act_route['route']:
          # Randomly decide whether to modify merchandise
          if random.choice([True, False]):
            # Select a random merchandise type
            modified_merchandise = random.choice(list(city_route['merchandise'].keys()))
            # Modify the quantity or replace the merchandise type
            if random.choice([True, False]):
              # Modify the quantity
              city_route['merchandise'][modified_merchandise] = random.randint(1, 5)
            else:
              # Replace the merchandise type
              new_merchandise = random.choice(merchandise_types)
              if modified_merchandise in city_route['merchandise']:
                city_route['merchandise'][new_merchandise] = city_route['merchandise'][modified_merchandise]
                del city_route['merchandise'][modified_merchandise]


        # Randomly modify merchandise
        #for city_route in act_route['route']:
            # Select random merchandise types
            #selected_merchandise = random.sample(merchandise_types, random.randint(1, len(merchandise_types)))
            # Assign random quantities to selected merchandise
            #city_route['merchandise'] = {item: random.randint(1, 5) for item in selected_merchandise}

        act_routes.append(act_route)

# Write the act routes list to a new JSON file
with open('act_routes.json', 'w') as file:
    json.dump(act_routes, file, indent=4)

In [7]:
# Download the actual routes
# from google.colab import files
# files.download('act_routes.json')

In [8]:
!pip install python-Levenshtein
import json
from collections import defaultdict
from Levenshtein import distance

# Function for calculating route edit distance similarity
def calculate_similarity(route1, route2):
    lev_distance = distance(route1, route2)
    max_length = max(len(route1), len(route2))
    similarity = 1 - lev_distance / max_length
    return similarity

# Function for calculating marchandise edit distance similarity
def calculate_merchandise_similarity(merch1, merch2):
    lev_distance = distance(merch1, merch2)
    max_length = max(len(merch1), len(merch2))
    similarity = 1 - lev_distance / max_length
    return similarity

# open standard route file
with open('routes.json', 'r') as file:
    routes = json.load(file)

# open actual route file
with open('act_routes.json', 'r') as file:
    act_routes = json.load(file)

# Create a dictionary to store the most similar route for each actual route
most_similar_routes = defaultdict(list)

# Correct prediction counter
correct_predictions = 0

# Iterate through all actual routes
for act_route in act_routes:
    act_route_id = act_route['id']
    act_route_str = ' '.join([f"{route['from']} {route['to']}" for route in act_route['route']])

    similarities = []

    # route similarity calculating
    for standard_route in routes:
        standard_route_id = standard_route['id']
        standard_route_str = ' '.join([f"{route['from']} {route['to']}" for route in standard_route['route']])

        route_similarity = calculate_similarity(standard_route_str, act_route_str)

        # merchandise similarity calculating
        merchandise_similarity = 0
        for i, route in enumerate(act_route['route']):
            act_merchandise = route['merchandise']
            standard_merchandise = standard_route['route'][0]['merchandise']
            merchandise_similarity += calculate_merchandise_similarity(str(standard_merchandise), str(act_merchandise))
        merchandise_similarity /= len(standard_route['route'])

        #total similarity calculating
        total_similarity = (route_similarity + merchandise_similarity) / 2
        similarities.append((standard_route_id, total_similarity))

    # sort the max similarity
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Find the most similar route
    max_similarity = similarities[0]

    # Store the most similar route for this actual route
    most_similar_routes[act_route_id].append(max_similarity)

    # If the actual route id is the same as the predicted route id, increment the correct prediction counter
    if act_route_id == max_similarity[0]:
        correct_predictions += 1

# Print the most similar routes for all actual routes
# for act_route_id, max_similarities in most_similar_routes.items():
#    for i, max_similarity in enumerate(max_similarities):
#        print(f"Max similarity for actual route {act_route_id}-{i+1}: Standard Route {max_similarity[0]} with similarity {max_similarity[1]}")

# Calculate and print the prediction accuracy
accuracy = correct_predictions / len(act_routes)
print(f"Prediction accuracy: {accuracy * 100}%")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.7 MB/s eta 0:00:00
Prediction accuracy: 99.2%


In [9]:
# Convert 'standard routes' file to Spark DataFrame
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

# Load data from JSON file
with open('routes.json', 'r') as file:
    data = json.load(file)

# Initialize an empty DataFrame to store the processed data
df = pd.DataFrame()

# Iterate over each element
for element in data:
  id = element['id']

# Iterate over routes
  for route in element['route']:
    from_city = route['from']
    to_city = route['to']

    # Iterate over merchandise
    for merchandise, quantity in route['merchandise'].items():
        # Append data to DataFrame
        df = df.append({
            'id': id,
            'from': from_city,
            'to': to_city,
            'merchandise': merchandise,
            'quantity': quantity
        }, ignore_index=True)
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert DataFrame to Spark DataFrame
st_routes_df = spark.createDataFrame(df)

In [10]:
from pyspark.sql.functions import expr

# Create a new column that is the merchandise column value repeated quantity times
str_df = st_routes_df.withColumn('merchandise_quantity', expr("repeat(merchandise, quantity)"))
str_df = str_df.drop('merchandise','quantity')

str_df.show(5)

+---+-----------+-----------+--------------------+
| id|       from|         to|merchandise_quantity|
+---+-----------+-----------+--------------------+
|  0|    Helmond|Schoonhoven|waterwaterwaterwater|
|  0|    Helmond|Schoonhoven|          honeyhoney|
|  0|    Helmond|Schoonhoven|                pens|
|  0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|  0|Schoonhoven|    Zaandam|               honey|
+---+-----------+-----------+--------------------+
only showing top 5 rows



In [11]:
# delete the duplicated routes
stpaths = str_df.dropDuplicates(["id", "from", "to"])

stpaths.show(5)

+---+------------+------------+--------------------+
| id|        from|          to|merchandise_quantity|
+---+------------+------------+--------------------+
|  0|     Helmond| Schoonhoven|waterwaterwaterwater|
|  0| Schoonhoven|     Zaandam|waterwaterwaterwater|
|  1|Klazienaveen|   The Hague|                pens|
|  1|      Nuenen|Klazienaveen|penspenspenspensp...|
|  2|    Enschede|   Harlingen|    milkmilkmilkmilk|
+---+------------+------------+--------------------+
only showing top 5 rows



In [19]:
#STANDERD ROUTE
from pyspark.sql.functions import concat_ws, collect_list

# Group the dataframe by 'id', 'from', and 'to' and concatenate the 'merchandise_quantity' strings
st_df = str_df.groupBy('id', 'from', 'to').agg(concat_ws('', collect_list('merchandise_quantity')).alias('combined_merchandise'))

st_df.show(5)

+---+------------+------------+--------------------+
| id|        from|          to|combined_merchandise|
+---+------------+------------+--------------------+
|  0|     Helmond| Schoonhoven|waterwaterwaterwa...|
|  0| Schoonhoven|     Zaandam|waterwaterwaterwa...|
|  1|Klazienaveen|   The Hague|penstomatoestomat...|
|  1|      Nuenen|Klazienaveen|penspenspenspensp...|
|  2|    Enschede|   Harlingen|milkmilkmilkmilkt...|
+---+------------+------------+--------------------+
only showing top 5 rows



In [12]:
import pandas as pd
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Load data from JSON file
with open('act_routes.json', 'r') as file:
    data = json.load(file)

# Initialize an empty DataFrame to store the processed data
df = pd.DataFrame()

# Dictionary to store the counter for each unique ID
id_counter = {}

# Iterate over each element
for element in data:
    id = element['id']

    # Increment the counter for each unique ID
    if id in id_counter:
        id_counter[id] += 1
    else:
        id_counter[id] = 1

    # Counter for each ID
    counter = id_counter[id]

    # Iterate over routes
    for route in element['route']:
        from_city = route['from']
        to_city = route['to']

        # Iterate over merchandise
        for merchandise, quantity in route['merchandise'].items():
            # Append data to DataFrame with modified ID
            df = df.append({
                'id': f"{id}-{counter}",  # Assign modified ID with counter
                'original_id': id,  # Store original ID
                'from': from_city,
                'to': to_city,
                'merchandise': merchandise,
                'quantity': quantity
            }, ignore_index=True)

# Convert DataFrame to Spark DataFrame
spark = SparkSession.builder.getOrCreate()
ac_routes_df = spark.createDataFrame(df)

# Assign sequential counts to duplicate IDs
ac_routes_df = ac_routes_df.withColumn('new_id', monotonically_increasing_id())


In [24]:
# combine merchandies and quantity
acr_df = ac_routes_df.withColumn('merchandise_quantity', expr("repeat(merchandise, quantity)"))
acr_df = acr_df.drop('merchandise', 'quantity', 'new_id')

acr_df.show(20)


+---+-----------+-----------+-----------+--------------------+
| id|original_id|       from|         to|merchandise_quantity|
+---+-----------+-----------+-----------+--------------------+
|0-1|          0|Barendrecht|Schoonhoven|waterwaterwaterwater|
|0-1|          0|Barendrecht|Schoonhoven|          honeyhoney|
|0-1|          0|Barendrecht|Schoonhoven|                pens|
|0-1|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|0-1|          0|Schoonhoven|    Zaandam|               honey|
|0-1|          0|Schoonhoven|    Zaandam|butterbutterbutte...|
|0-2|          0|     Bergen|Schoonhoven|waterwaterwaterwater|
|0-2|          0|     Bergen|Schoonhoven|                pens|
|0-2|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|0-2|          0|Schoonhoven|    Zaandam|               honey|
|0-2|          0|Schoonhoven|    Zaandam|    penspenspenspens|
|0-3|          0|    Helmond|Schoonhoven|          honeyhoney|
|0-3|          0|    Helmond|Schoonhoven|              

In [25]:
#ACTUAL ROUTE
from pyspark.sql.functions import first

# Group the dataframe by 'id', 'original_id', 'from', and 'to' and select the first 'merchandise_quantity' value
ac_df = acr_df.groupby('id', 'original_id', 'from', 'to').agg(first('merchandise_quantity').alias('combined_merchandise'))

ac_df.show()

+----+-----------+-----------+-----------+--------------------+
|  id|original_id|       from|         to|combined_merchandise|
+----+-----------+-----------+-----------+--------------------+
| 0-1|          0|Barendrecht|Schoonhoven|waterwaterwaterwater|
| 0-1|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|0-10|          0|    Helmond|Schoonhoven|waterwaterwaterwater|
|0-10|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
| 0-2|          0|     Bergen|Schoonhoven|waterwaterwaterwater|
| 0-2|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
| 0-3|          0|    Helmond|Schoonhoven|          honeyhoney|
| 0-3|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
| 0-4|          0|    Helmond|Schoonhoven|waterwaterwaterwater|
| 0-4|          0|Schoonhoven|    Zaandam|     waterwaterwater|
| 0-5|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
| 0-5|          0|  Staphorst|Schoonhoven|waterwaterwaterwater|
| 0-6|          0|    Helmond|Schoonhove

In [14]:
# delete the duplicated routes
acpaths = acr_df.dropDuplicates(["id", "original_id", "from", "to"])

acpaths.show(5)

+----+-----------+-----------+-----------+--------------------+
|  id|original_id|       from|         to|merchandise_quantity|
+----+-----------+-----------+-----------+--------------------+
| 0-1|          0|Barendrecht|Schoonhoven|waterwaterwaterwater|
| 0-1|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
|0-10|          0|    Helmond|Schoonhoven|waterwaterwaterwater|
|0-10|          0|Schoonhoven|    Zaandam|waterwaterwaterwater|
| 0-2|          0|     Bergen|Schoonhoven|waterwaterwaterwater|
+----+-----------+-----------+-----------+--------------------+
only showing top 5 rows



In [15]:
# Rename the columns
pa_standard = stpaths.selectExpr("id as st_id", "from as st_from", "to as st_to")
pa_actual = acpaths.selectExpr("id as act_id", "original_id", "from as act_from", "to as act_to")

pa_actual.show()

df_joined = pa_actual.join(pa_standard, (pa_actual.original_id == pa_standard.st_id), 'outer')

df_joined.show(5)

+------+-----------+---------------+-------------------+
|act_id|original_id|       act_from|             act_to|
+------+-----------+---------------+-------------------+
|   1-8|          1|   Klazienaveen|          The Hague|
|  13-3|         13|         Raalte|   Koog aan de Zaan|
|  23-7|         23|        Tilburg|           Schinnen|
|  32-1|         32|    Heerlerbaan|             Leiden|
| 40-10|         40|       Hillegom|        Heerlerbaan|
|  77-6|         77|    Oosterwolde|         Oegstgeest|
|  82-1|         82|        Leusden|           Ter Apel|
|  89-9|         89|        Naarden|             Borger|
|  94-8|         94|        Helmond|              Wezep|
|135-10|        135|     Teijlingen|          Zandvoort|
| 153-7|        153|          Zeist|Hendrik-Ido-Ambacht|
| 160-6|        160|        Brummen|        Dedemsvaart|
| 173-3|        173|     Schaesberg|                Oss|
| 181-7|        181|      Landgraaf|       Monnickendam|
| 208-5|        208|     Woudri

In [18]:
from pyspark.sql import SparkSession
import json
from collections import defaultdict
from itertools import chain

spark = SparkSession.builder.getOrCreate()

# Load the json file
with open('act_routes.json', 'r') as f:
    data = json.load(f)

# Process the data
processed_data = defaultdict(list)
for entry in data:
    original_id = entry['id']
    for i, route in enumerate(entry['route']):
        from_city = route['from']
        to_city = route['to']
        merchandise_list = []
        for merchandise, quantity in route['merchandise'].items():
            merchandise_list.extend([merchandise]*quantity)
        processed_data[original_id].extend([f"{original_id}-{i}", from_city, to_city, *merchandise_list])

# Convert dictionary to list of tuples
processed_data = [tuple(chain(*v)) for v in processed_data.values()]

# Create DataFrame
df = spark.createDataFrame(processed_data)

# Show DataFrame
df.show()


Py4JJavaError: ignored

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from Levenshtein import distance

def levenshtein_distance(s1, s2):
    return distance(s1, s2)

levenshtein_distance_udf = udf(levenshtein_distance, IntegerType())
